**Business News Sentiment Classifier**

*   **Objective**: fine-tune the *microsoft/deberta-v3-large* to classify business news headlines into 3 categories
  *   positive
  *   neutral
  *   negative
*   Resource
  *   [Model](https://huggingface.co/microsoft/deberta-v3-large)

## Environment Setup

*   Install Libraries

In [1]:
!pip install transformers datasets evaluate wandb scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

*   Import Libraries

In [2]:
import re
import numpy as np
import pandas as pd
import torch
import evaluate
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    DataCollatorWithPadding,
    set_seed
)
import wandb

*   Set Random Seed for Reproducibility

In [3]:
SEED = 42
set_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Tracking

*   Initialize Wandb

In [4]:
wandb.init(
    project="deberta-v3-large_business_news_sentiment_classifier",
    config={
        "model": "microsoft/deberta-v3-large",
        "seed": SEED,
        "batch_size": 8,
        "learning_rate": 6e-6,
        "num_train_epochs": 3,
        "dataset_size": 136995
    }
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: logicalqubit (logical-qubit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Data Prepartion

*   Define Label Mapping

In [5]:
LABEL_MAP = {
    "positive": 0,
    "neutral": 1,
    "negative": 2
}
ID2LABEL = {v: k for k, v in LABEL_MAP.items()}

*   Load Dataset

In [6]:
df = pd.read_csv("cleaned_business_news_sentiment.csv")

*   Process Filtering and Mapping

In [7]:
df = df[df['sentiment'].isin(LABEL_MAP.keys())]
df['sentiment'] = df['sentiment'].map(LABEL_MAP).astype(int)

*   Process Sampling

In [8]:
SAMPLES_PER_LABEL = 45665
df = (
    df.groupby('sentiment', group_keys=False)
    .apply(lambda x: x.sample(n=SAMPLES_PER_LABEL, random_state=SEED))
    .reset_index(drop=True)
)

<ipython-input-8-a59c1942284e>:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=SAMPLES_PER_LABEL, random_state=SEED))


In [9]:
print(f"Dataset Scale: {len(df)}")
print(f"Sentiment Distribution:\n{df['sentiment'].value_counts()}")

Dataset Scale: 136995
Sentiment Distribution:
sentiment
0    45665
1    45665
2    45665
Name: count, dtype: int64


*   Split Dataset

In [10]:
train_val, test_df = train_test_split(
    df,
    test_size=0.1,
    stratify=df['sentiment'],
    random_state=SEED
)

train_df, val_df = train_test_split(
    train_val,
    test_size=0.1,
    stratify=train_val['sentiment'],
    random_state=SEED
)

In [11]:
print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

Train: 110965, Validation: 12330, Test: 13700


*   Convert pandas DataFrame to Hugging Face DatasetDict

In [12]:
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df, preserve_index=False),
    "validation": Dataset.from_pandas(val_df, preserve_index=False),
    "test": Dataset.from_pandas(test_df, preserve_index=False)
})

# Tokenizing

*   Initialize Model & Tokenizer

In [13]:
MODEL_NAME = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.model_max_length = 256

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


*   Define Tokenization Function

In [14]:
def tokenize_fn(batch):
    return tokenizer(
        batch["title"],
        truncation=True,
        max_length=256,
        padding=False,
        add_special_tokens=True
    )

*   Tokenize Dataset & Format for PyTorch

In [15]:
tokenized_ds = dataset.map(
    tokenize_fn,
    batched=True,
    batch_size=2048,
    remove_columns=["title"],
    num_proc=8
)
tokenized_ds = tokenized_ds.rename_column("sentiment", "labels")

Map (num_proc=8):   0%|          | 0/110965 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/12330 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/13700 [00:00<?, ? examples/s]

# Model Initialization

*   Load Pretrained Model

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    id2label=ID2LABEL,
    label2id=LABEL_MAP
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


*   Set Training Arguments

In [17]:
training_args = TrainingArguments(
    output_dir="./deberta-v3-large_business_news_sentiment_classifier-checkpoints",
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=20,

    eval_strategy="steps",
    do_eval=True,
    eval_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,

    learning_rate=6e-6,
    weight_decay=0.01,
    warmup_steps=300,
    lr_scheduler_type="linear",
    optim="adamw_torch_fused",

    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,

    fp16=True,
    bf16=False,
    max_grad_norm=1.0,

    logging_steps=1000,
    report_to="wandb",

    seed=SEED,
    dataloader_num_workers=8
)

# Metrics & Trainer

*   Define Metric Functions

In [18]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

*   Initialize Trainer

In [19]:
data_collator = DataCollatorWithPadding(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=15)],
)

# Train & Evaluate

In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Accuracy,F1
1000,0.466400,0.216541,0.959367,0.959372
2000,0.180100,0.173803,0.970560,0.970571
3000,0.123400,0.153648,0.974858,0.974901
4000,0.125000,0.145937,0.976075,0.976084
5000,0.125400,0.092853,0.980454,0.980456
6000,0.113500,0.091413,0.982806,0.982811
7000,0.107100,0.068909,0.984915,0.984917
8000,0.097700,0.109545,0.981346,0.981346
9000,0.092100,0.090660,0.983049,0.983034
10000,0.093500,0.072686,0.985888,0.985895


TrainOutput(global_step=41613, training_loss=0.060704488479030864, metrics={'train_runtime': 13444.5878, 'train_samples_per_second': 24.761, 'train_steps_per_second': 3.095, 'total_flos': 2.1259149660644616e+16, 'train_loss': 0.060704488479030864, 'epoch': 3.0})

*   Evaluate by Validation

In [21]:
val_results = trainer.evaluate(tokenized_ds["validation"])
print(f"Validation Accuracy: {val_results['eval_accuracy']:.4f}")
print(f"Validation F1: {val_results['eval_f1']:.4f}")

Validation Accuracy: 0.9904
Validation F1: 0.9904


*   Evaluate by Test

In [22]:
test_results = trainer.evaluate(tokenized_ds["test"])
print(f"Test Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"Test F1: {test_results['eval_f1']:.4f}")

Test Accuracy: 0.9896
Test F1: 0.9896


*   Save Model

In [23]:
trainer.save_model("deberta-v3-large-business-news-sentiment-classifier")
tokenizer.save_pretrained("deberta-v3-large-business-news-sentiment-classifier")

('deberta-v3-large-business-news-sentiment-classifier/tokenizer_config.json',
 'deberta-v3-large-business-news-sentiment-classifier/special_tokens_map.json',
 'deberta-v3-large-business-news-sentiment-classifier/spm.model',
 'deberta-v3-large-business-news-sentiment-classifier/added_tokens.json',
 'deberta-v3-large-business-news-sentiment-classifier/tokenizer.json')

*   Finish Wandb

In [24]:
wandb.finish()

eval/accuracy,▁▄▄▅▆▆▇▆▆▇▇▇▇▆▇▇▇▇▇▇▇▇██████████████████
eval/f1,▁▄▄▅▆▆▇▆▆▇▇▇▇▆▇▇▇▇▇▇▇▇██████████████████
eval/loss,█▆▅▅▂▂▁▃▂▁▂▁▂▄▂▁▂▃▁▂▂▂▁▁▁▁▂▂▂▂▂▂▂▂▂▁▂▂▁▂
eval/runtime,▁▁▁▂▁▂▁▁▂▂▁▁▁▂▂▂▂▁▁▂▂▂▂▁▂▁▂▂▂▂▁▁▂▁▂▂▂▂▂█
eval/samples_per_second,▆█▇▆▇▅█▇▅▅▇▇▆▅▅▄▅▆▆▅▅▄▅▆▅▆▆▆▅▄▆▆▄▇▆▅▅▃▃▁
eval/steps_per_second,▆█▇▆▇▅█▇▅▅▇▇▆▅▆▄▅▆▆▅▅▄▅▆▅▆▆▆▅▄▆▆▄▇▆▅▅▃▃▁
train/epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████
train/global_step,▁▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
train/grad_norm,▁▁▃▁▁▄▁▁▂▁▁▁▁▁▁▂▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁
train/loss,█▄▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
